In [1]:
# using spacy to implement NER to identify entities in report
import os
import spacy
import pdfplumber
import numpy as np
import pandas as pd
import xlsxwriter
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from pikepdf import Pdf
from spacy import displacy
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')
import textrazor

textrazor.api_key = '706537941648c9e8d8c9e99fba2bc0119d8150883b38ceb21b04c2fd'
# db688@cornell.edu: '706537941648c9e8d8c9e99fba2bc0119d8150883b38ceb21b04c2fd'
# diyabansal2002@gmail.com: '5b9f3f80983ef72fc69768f28a97225b44c17b7669d954ab41eab2c3'
client = textrazor.TextRazor(extractors=["entities", "topics"])

NER = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/diyabansal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/diyabansal/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/diyabansal/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/diyabansal/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
# RETURNS A LIST OF STRINGS WITH RANGE OF PAGE NUMBERS TO BE PARSED THROUGH

def pages_list(start, end):
    pages = []
    current = start
    
    while (current <= end):
        page = '<Page:' + str(current) + '>'
        pages.append(page)
        current = current + 1
        
    return pages

In [3]:
# CONVERTS CID ENCODED WORDS TO CHARACTERS

def replace_cid_space(text_str):
    while ('cid' in text_str.lower()):
    
        # finding the (cid:XXXX) parts of the strings
        starting_index = text_str.index('(cid:')
        ending_index = text_str.index(')', starting_index)
        cid_word = text_str[starting_index:ending_index+1]
    
        # replacing (cid:XXXX) with ''
        text_str = text_str.replace(cid_word, '')
    
    return text_str
    

In [4]:
# apply word tokenization and part-of-speech tagging to the sentence

def preprocess_nltk(sen):
    sen = nltk.word_tokenize(sen)
    sen = nltk.pos_tag(sen)
    return sen

In [41]:
# to check if parameter string has anything other than 'nan', spaces, and commas
# returns a boolean

def contains_nan(str):
    str = str.replace('nan', '')
    str = str.replace(' ', '')
    str = str.replace(',', '')
    if (str == ''):
        return True
    else:
        return False

In [42]:
# returns a string replacing all spaces and occurences of the word 'nan'

def remove_nan(str):
    str = str.replace('nan', '')
    str = str.replace(' ', '')
    return str

In [19]:
x0 = 0    # Distance of left side of character from left side of page.
x1 = 0.5  # Distance of right side of character from left side of page.
y0 = 0    # Distance of bottom of character from bottom of page.
y1 = 1    # Distance of top of character from bottom of page.

all_content = []
start = int(input('Enter range of pages: starting page: '))
end = int(input('Enter range of pages: ending page: '))
print()
pages = pages_list(start, end)

df = pd.DataFrame(columns=['Sentence','Entity', 'SpaCy Label', 'NLTK Label', 'TextRazor Label', 'TextRazor Confidence Score'])

with pdfplumber.open("annual-report-2019.pdf") as pdf:
    
    # for each page
    for i, page in enumerate(pdf.pages): 
        
        if (str(page) in pages):
            width = page.width
            height = page.height

            # Crop page
            left_bbox = (x0*float(width), y0*float(height), x1*float(width), y1*float(height))
            page_crop = page.crop(bbox=left_bbox)
            left_text = page_crop.extract_text()

            left_bbox = (0.5*float(width), y0*float(height), 1*float(width), y1*float(height))
            page_crop = page.crop(bbox=left_bbox)
            right_text = page_crop.extract_text()
        
            page_context = ' '.join([left_text, right_text])
            all_content.append(page_context)

            # text1= NER(replace_cid_space(page_context))
            # sentences = list(text1.sents)
            
            # for each sentence in all sentences on a page
            
            # for sentence in sentences:
            for sentence in nltk.sent_tokenize(replace_cid_space(page_context)):
                
                # using SPACY
                text2 = NER(str(sentence))
                
                # removing '\n' from each sentence
                sen = str(sentence)
                sen = sen.replace('\n','')
                
                # parsing through each word
                for word in text2.ents:
                    newrow = pd.DataFrame({'Sentence': [sen], 'Entity':[word.text.upper()], 'SpaCy Label':[word.label_]}, columns=['Sentence','Entity', 'SpaCy Label', 'NLTK Label', 'TextRazor Label', 'TextRazor Confidence Score'])
                    df = pd.concat([df, newrow], ignore_index = True)
                    
                # using NLTK
                for sent in nltk.sent_tokenize(sen):
                    for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
                        if hasattr(chunk, 'label'):
                            nltk_label = chunk.label()
                            nltk_entity = ' '.join(c[0] for c in chunk)
                            nltk_entity = nltk_entity.strip()
                            row = df.loc[::-1][df['Entity'] == nltk_entity][df['Sentence'] == sen] # finding a row with this entity and sentence
                            if (row.empty): # if no row found, creating a new row
                                row = pd.DataFrame({'Sentence': [sen], 'Entity':[nltk_entity.upper()], 'NLTK Label':[nltk_label]}, columns=['Sentence','Entity', 'SpaCy Label', 'NLTK Label', 'TextRazor Label', 'TextRazor Confidence Score'])
                                df = pd.concat([df, row], ignore_index = True)
                            else: # if row found, then appending the row with NLTK label
                                for index in row.index:
                                    row.loc[index,'NLTK Label'] = nltk_label
                                # making changes to original data frame
                                df.loc[row.index,:] = row[:]
                
                # REMOVING DUPLICATES AFTER SPACY & NLTK
                df = df.drop_duplicates() # removing duplicates
                df = df.drop_duplicates(subset=['Sentence', 'Entity', 'SpaCy Label'], keep = 'first') # removing duplicates with same sentence entities and spacy label
                
                # USING TEXTRAZOR
                response = client.analyze(sen)
                entities = list(response.entities())
                seen = set()
                
                for entity in entities:
                    if(entity.id not in seen):
                        tr_fbt = str(entity.freebase_types)
                        tr_cf = float(entity.confidence_score)
                        
                        if (tr_fbt == '[]'):
                            tr_fbt = np.NaN
                        if (tr_cf == 0.5):
                            tr_cf = np.NaN
                        
                        row = df.loc[::-1][df['Entity'] == entity.id][df['Sentence'] == sen] # finding a row with this entity and sentence
                        # print(row)
                        
                        if (row.empty): # if no row found, creating a new row
                            row = pd.DataFrame({'Sentence': [sen], 'Entity':[entity.id.upper()], 'TextRazor Label':[tr_fbt], 'TextRazor Confidence Score':[tr_cf]}, columns=['Sentence','Entity', 'SpaCy Label', 'NLTK Label', 'TextRazor Label', 'TextRazor Confidence Score'])
                            # print(row)
                            df = pd.concat([df, row], ignore_index = True)
                        else: # if row found, then appending the row with NLTK label
                            for index in row.index:
                                row.loc[index,'TextRazor Label'] = tr_fbt
                                row.loc[index,'TextRazor Confidence Score'] = tr_cf
                            # print(row)
                            # making changes to original data frame
                            df.loc[row.index,:] = row[:]
                        seen.add(entity.id)
                        
                # REMOVING DUPLICATES AFTER TEXTRAZOR
                df = df.drop_duplicates() # removing duplicates
                # df = df.drop_duplicates(subset=['Sentence', 'Entity', 'SpaCy Label', 'NLTK Label']) # removing duplicates with same sentence entities and spacy nltk column values
                
            
# df = df.groupby(['Sentence','Entity'])
# df = df.groupby(['Sentence','Entity'])['SpaCy Label'].apply(','.join).reset_index()
df = df.reset_index()
df = df.drop(['index'], axis=1)
df = df.sort_values(by=['Entity'])
df

# displacy.render(text1,style="ent",jupyter=True)

Enter range of pages: starting page: 18
Enter range of pages: ending page: 21



,Sentence,Entity,SpaCy Label,NLTK Label,TextRazor Label,TextRazor Confidence Score
337,Named as a Leader in ...,0,CARDINAL,NaN,NaN,NaN
325,Named as a Leader in ...,1,MONEY,NaN,NaN,NaN
103,NIIT Technologies’ comme...,10,CARDINAL,NaN,NaN,NaN
159,18 Celebration of 15 years of partnership wit...,15,NaN,NaN,NaN,NaN
158,18 Celebration of 15 years of partnership wit...,15 YEARS,DATE,NaN,NaN,NaN
...,...,...,...,...,...,...
329,Named as a Leader in ...,WHITELANE RESEARCH,ORG,NaN,NaN,NaN
3,The Year Gone By FY 2019NI...,YEAR,DATE,NaN,NaN,NaN
5,The Year Gone By FY 2019NI...,YEAR GONE BY,NaN,ORGANIZATION,NaN,NaN
306,Incessant Technologies cited as a Strong ...,ZINNOV,PERSON,NaN,NaN,NaN


In [39]:
# creating a map for SpaCy and NLTK
# organized as Spacy and NLTK labels concatenated
map_list = ['CARDINAL',
 'DATE',
 'EVENT',
 'FACFACILITY',
 'GPEGPE',
 'LANGUAGE',
 'LAW',
 'LOCLOCATION',
 'MONEY',
 'NORPGSP',
 'ORDINAL',
 'ORGORGANIZATION',
 'PERCENT',
 'PERSONPERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART']

dict_label = {'CARDINAL': 'CARDINAL', 'DATE': 'DATE', 'EVENT':'EVENT', 'FAC':'FACILITY', 'FACILITY':'FACILITY',
              'GPE':'GEO-POLITICAL ENTITY', 'LANGUAGE':'LANGUAGE', 'LAW':'LAW', 'LOC':'LOCATION', 'LOCATION':'LOCATION',
             'MONEY':'MONEY', 'NORP':'NATIONAL, RELIGIOUS OR POLITICAL GROUPS', 'GSP':'GEO-SOCIO-POLITICAL ENTITY',
             'ORDINAL':'ORDINAL', 'ORG':'ORGANIZATION', 'ORGANIZATION':'ORGANIZATION','PERCENT':'PERCENT', 'PERSON':'PERSON',
             'PRODUCT':'PRODUCT', 'QUANTITY':'QUANTITY', 'TIME':'TIME', 'WORK_OF_ART':'WORK OF ART'}

# NLTK: FACILITY, GPE, GSP, LOCATION, ORGANIZATION, PERSON

In [57]:
list_iter = [] # list keeping track of entities we already have parsed through

# new dataframe which stores single entity entries without sentences
new_df = pd.DataFrame(columns=['Sentence','Entity', 'SpaCy Label', 'NLTK Label', 'TextRazor Label', 'TextRazor Confidence Score'])

for index, row in df.iterrows():
    
    ele = row['Sentence'] + row['Entity']
    
    if (not ele in list_iter):

        list_iter.append(ele)
        sub = df.loc[(df['Entity'] == row['Entity']) & (df['Sentence'] == row['Sentence'])]

        spa = ''
        nlt = ''
        tl = ''
        tc = ''

        for index2, row2 in sub.iterrows():

            spa2 = row2['SpaCy Label']
            nlt2 = row2['NLTK Label']
            tl2 = row2['TextRazor Label']
            tc2 = row2['TextRazor Confidence Score']

            spa = spa + ' ' + str(spa2)
            nlt = nlt + ' ' + str(nlt2)
            tl = tl + ' ' + str(tl2)
            tc = tc + ' ' + str(tc2)

        spa = spa[1:]
        nlt = nlt[1:]
        tl = tl[1:]
        tc = tc[1:]

        new_row = pd.DataFrame({'Sentence':[row['Sentence']], 'Entity':[row['Entity']], 'SpaCy Label': [spa], 'NLTK Label': [nlt], 'TextRazor Label':[tl], 'TextRazor Confidence Score':[tc]}, columns=['Sentence', 'Entity', 'SpaCy Label', 'NLTK Label', 'TextRazor Label', 'TextRazor Confidence Score'])
        new_df = pd.concat([new_df, new_row], ignore_index = True)

new_df

,Sentence,Entity,SpaCy Label,NLTK Label,TextRazor Label,TextRazor Confidence Score
0,Named as a Leader in ...,0,CARDINAL,nan,nan,nan
1,Named as a Leader in ...,1,MONEY,nan,nan,nan
2,NIIT Technologies’ comme...,10,CARDINAL,nan,nan,nan
3,18 Celebration of 15 years of partnership wit...,15,nan,nan,nan,nan
4,18 Celebration of 15 years of partnership wit...,15 YEARS,DATE,nan,nan,nan
...,...,...,...,...,...,...
314,"Acquired WHISHWORKS with WHISHWORKS, a Mul...",WHISHWORKS,nan ORG,nan ORGANIZATION,['/business/employer'] nan,1.748 nan
315,Named as a Leader in ...,WHITELANE RESEARCH,nan nan ORG,ORGANIZATION nan nan,nan ['/organization/organization'] nan,nan nan nan
316,The Year Gone By FY 2019NI...,YEAR,DATE,nan,nan,nan
317,The Year Gone By FY 2019NI...,YEAR GONE BY,nan,ORGANIZATION,nan,nan


In [86]:
# if any 2 labels match, identified as true

final_df = pd.DataFrame(columns=['Sentence', 'Entity', 'SpaCy Label', 'NLTK Label', 'TextRazor Label', 'TextRazor Confidence Score', 'Final Label'])
for index, row in new_df.iterrows():
    
    final_label = ''
    
    # removing duplicate words
    spa = ' '.join(dict.fromkeys(str(row['SpaCy Label']).split()))
    nlt = ' '.join(dict.fromkeys(str(row['NLTK Label']).split()))
    tr = ' '.join(dict.fromkeys(str(row['TextRazor Label']).split()))
    tc = ' '.join(dict.fromkeys(str(row['TextRazor Confidence Score']).split()))
    
    # removing nan from string
    spa = remove_nan(spa)
    nlt = remove_nan(nlt)
    tr = remove_nan(tr)
    tr = tr.upper()
    tc = remove_nan(tc)
    
    map_compare = spa + nlt
    
    if ((map_compare in map_list) and (not nlt == '')):
        final_label = dict_label[spa]

    elif (not spa == ''):
        if (dict_label[spa] in tr):
            final_label = dict_label[spa]
        
    elif (not nlt == ''):
        if (dict_label[nlt] in tr):
            final_label = dict_label[nlt]
    
    spa = ' '.join(dict.fromkeys(str(row['SpaCy Label']).split()))
    nlt = ' '.join(dict.fromkeys(str(row['NLTK Label']).split()))
    tr = ' '.join(dict.fromkeys(str(row['TextRazor Label']).split()))
    tr = tr.lower()
    tc = ' '.join(dict.fromkeys(str(row['TextRazor Confidence Score']).split()))
    
    final_row = pd.DataFrame({'Sentence':[row['Sentence']], 'Entity':[row['Entity']], 'SpaCy Label': [spa], 'NLTK Label': [nlt], 'TextRazor Label':[tr], 'TextRazor Confidence Score':[tc], 'Final Label': [final_label]}, columns=['Sentence', 'Entity', 'SpaCy Label', 'NLTK Label', 'TextRazor Label', 'TextRazor Confidence Score', 'Final Label'])
    final_df = pd.concat([final_df, final_row], ignore_index = True)

final_df

,Sentence,Entity,SpaCy Label,NLTK Label,TextRazor Label,TextRazor Confidence Score,Final Label
0,Named as a Leader in ...,0,CARDINAL,nan,nan,nan,
1,Named as a Leader in ...,1,MONEY,nan,nan,nan,
2,NIIT Technologies’ comme...,10,CARDINAL,nan,nan,nan,
3,18 Celebration of 15 years of partnership wit...,15,nan,nan,nan,nan,
4,18 Celebration of 15 years of partnership wit...,15 YEARS,DATE,nan,nan,nan,
...,...,...,...,...,...,...,...
314,"Acquired WHISHWORKS with WHISHWORKS, a Mul...",WHISHWORKS,nan ORG,nan ORGANIZATION,['/business/employer'] nan,1.748 nan,ORGANIZATION
315,Named as a Leader in ...,WHITELANE RESEARCH,nan ORG,ORGANIZATION nan,nan ['/organization/organization'],nan,ORGANIZATION
316,The Year Gone By FY 2019NI...,YEAR,DATE,nan,nan,nan,
317,The Year Gone By FY 2019NI...,YEAR GONE BY,nan,ORGANIZATION,nan,nan,


In [84]:
# selecting all rows that are valid

final_df = final_df.loc[final_df['Final Label'] != '']
final_df

,Sentence,Entity,SpaCy Label,NLTK Label,TextRazor Label,TextRazor Confidence Score,Final Label
37,NIIT Technologies’ comme...,ADVANTAGEGO,nan ORG,ORGANIZATION nan,nan,nan,ORGANIZATION
88,"19 In this role, Sanjay has been actively supp...",COMMITTEE,ORG nan,nan ORGANIZATION,nan,nan,ORGANIZATION
102,"Furthermore, as part of this strategy, your co...",DIGITAL,ORG nan,nan ORGANIZATION,nan,nan,ORGANIZATION
103,"Acquired WHISHWORKS with WHISHWORKS, a Mul...",DIGITAL,nan ORG,ORGANIZATION nan,nan,nan,ORGANIZATION
107,"Acquired WHISHWORKS with WHISHWORKS, a Mul...",DIGITAL INTEGRATION,nan ORG,ORGANIZATION nan,nan,nan,ORGANIZATION
109,Incessant Technologies cited as a Strong ...,DIGITAL SERVICES,ORG nan,nan ORGANIZATION,nan,nan,ORGANIZATION
124,The company also introduced Exact Max to impro...,EXACT MAX,nan PERSON,PERSON nan,nan,nan,PERSON
125,Sanjay Mal appointed as Sanjay brings over ...,EXECUTIVE,ORG nan,nan ORGANIZATION,nan,nan,ORGANIZATION
129,"During the year, your company: Collaborated w...",FINANCIAL SERVICES,ORG nan,nan ORGANIZATION,nan,nan,ORGANIZATION
139,Named as a Leader in ...,GRANT THORNTON0,nan,PERSON nan,nan ['/people/person'],nan,PERSON


In [87]:
try:
    
    # writing to excel file
    final_df.to_excel("output.xlsx", sheet_name='Sheet 1', index=False)
    print()
    print('Data exported to excel sheet named "output.xlsx" successfully.')
    
except Exception:
    print('There was an error!')


Data exported to excel sheet named "output.xlsx" successfully.
